In [137]:
import os
os.chdir('/users/jinyinzhe/downloads/')
import pandas as pd
pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, urlretrieve
import re
import pytesseract

In [138]:
web1 = 'http://sh.ziroom.com/z/p1/'
req1 = Request(web1, headers={'User-Agent': 'Mozilla/5.0'})
downloaded_web1 = urlopen(req1).read()
house1 = BeautifulSoup(downloaded_web1)
house_price1 = []
get_price_code1=house1.find('div',class_='Z_list-box').find_all('span',class_='num')
for price_code1 in get_price_code1:
    house_price1.append(price_code1.get('style'))
position_list1 = [re.findall(r'background-position: -(.+)px',i) for i in house_price1]
price_list1 = [int(float(j[0])/21.4+1) for j in position_list1]
img_url1 = "http:" + re.findall(r'background-image: url\((.*?)\)',house_price1[0])[0]
urlretrieve(img_url1,'background_img1.png')
text1 = pytesseract.image_to_string('background_img1.png',lang="eng",config='--psm 6')
nums1 = text1[:10]
num1 = [i for i in nums1]
price1 = [num1[i-1] for i in price_list1]

In [139]:
text1

'9310867542\n\x0c'

In [140]:
digit=house1.find_all('div',class_='item')

In [141]:
for i in range(len(digit)):
    if 'data-location' not in str(digit[i]):
        del digit[i]

In [142]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=option)
driver.get('http://sh.ziroom.com/z/p1/')

In [143]:
house_price = []
house_name = []
house_square = []
house_floor = []
house_distance = []
j = 0
for i in range(30):
    if i == 4:
        continue
    else:
        l = len(digit[i].find_all('span',class_='num'))
        price = ''.join(price1[j:j+l])
        j = j+l
        name_xpath = driver.find_element_by_xpath('/html/body/section/div[3]/div[2]/div['+str(i+1)+']/div[2]/h5/a')
        name = name_xpath.text
        square_floor_xpath = driver.find_element_by_xpath('/html/body/section/div[3]/div[2]/div['+str(i+1)+']/div[2]/div[1]/div[1]')
        square = square_floor_xpath.text.split('|')[0]
        floor = square_floor_xpath.text.split('|')[1]
        distance_xpath = driver.find_element_by_xpath('/html/body/section/div[3]/div[2]/div['+str(i+1)+']/div[2]/div[1]/div[2]')
        distance = distance_xpath.text
        house_price.append(price)
        house_name.append(name)
        house_square.append(square)
        house_floor.append(floor)
        house_distance.append(distance)

In [144]:
house_price_df = pd.DataFrame(house_price)
house_name_df = pd.DataFrame(house_name)
house_square_df = pd.DataFrame(house_square)
house_floor_df = pd.DataFrame(house_floor)
house_distance_df = pd.DataFrame(house_distance)
page1 = pd.concat([house_price_df,house_name_df,house_square_df,house_floor_df,house_distance_df],axis=1)
page1

,0,0,0,0,0
0,3560,合租·地杰国际城F欧香四季4居室-南卧,18㎡,14/18层,小区距御桥站步行约191米
1,9460,整租·华升公寓2室1厅-南,96.41㎡,20/34层,小区距江浦路站步行约150米
2,3160,合租·万里雅筑3居室-南卧,20.13㎡,14/16层,小区距上海西站站步行约325米
3,11330,整租·半岛花园2室1厅-南,100.31㎡,7/23层,小区距镇坪路站步行约542米
4,3290,合租·博爱家园4居室-南卧,17.5㎡,6/6层,小区距高科西路站步行约234米
5,3030,合租·紫叶花园东园4居室-南卧,17.5㎡,5/6层,小区距北蔡站步行约216米
6,3990,合租·仁和苑4居室-南卧,17.41㎡,7/12层,小区距江湾体育场站步行约243米
7,3490,合租·明园森林都市4居室-西卧,19.7㎡,24/34层,小区距汶水路站步行约686米
8,3290,合租·景泰大楼3居室-南卧,18.2㎡,11/18层,小区距临平路站步行约444米
9,2630,合租·日月新苑4居室-南卧,19㎡,3/6层,小区距上南路站步行约197米
